### Iniciar un programa con Spark

Para empezar a trabajar con Spark, necesitamos forzosamente crear una SparkSession. Además, si no lo hemos hecho ya, es necesario installar e importar la librería `pyspark`.



In [1]:
!pip install pyspark==3.4.0

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/310.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/310.8 MB 1.3 MB/s eta 0:04:03
     -------------------------------------- 0.1/310.8 MB 656.4 kB/s eta 0:07:54
     ---------------------------------------- 0.1/310.8 MB 1.1 MB/s eta 0:04:46
     ---------------------------------------- 0.2/310.8 MB 1.3 MB/s eta 0:03:58
     ---------------------------------------- 0.3/310.8 MB 1.4 MB/s eta 0:03:42
     ---------------------------------------- 0.5/310.8 MB 2.0 MB/s eta 0:02:38
     ---------------------------------------- 0.9/310.8 MB 2.7 MB/s eta 0:01:53
     ---------------------------------------- 1.6/310.8 MB 4.0 MB/s eta 0:01:18
     ---------------------------------------- 2.1/310.8 MB 4.7 MB/s eta 0:01:07
     ---------------------------------------- 2.7/310.8 MB 5.4 MB/s eta 0:00:57
     ---------------------------------------- 3.2

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

Vamos a trabajar con datos del Covid-19 en México. Revísalos y descárgalos de la siguiente página web:

https://www.kaggle.com/datasets/tavoglc/covid19-data-from-mexico?resource=download

In [3]:
df = spark.read.csv('C:\\Users\\Bertani\\Downloads\\covidmex.csv',header=True)
df.show(5)

+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+
|ID_REGISTRO|ENTIDAD_RES|MUNICIPIO_RES|FECHA_INGRESO|FECHA_SINTOMAS|covidt|delta|               lat|               long|               alt|                 qry|dayofyear|       lengthofday|
+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+
|     z526b3|          9|           12|   2020-12-21|    2020-12-18|     1|    3| 19.20155345588235| -99.20180252450989|3008.9460784313724|Cve_Ent==9 & Cve_...|      356|11.768371962176587|
|     z3d1e2|          9|            5|   2020-04-22|    2020-04-20|     1|    2|         19.482945|         -99.113471|            2229.0|Cve_Ent==9 & Cve_...|      113|13.441805501598786|
|     z21f6f|          7|            9|   2020-04-

### Análisis exploratorio usando SparkDataFrames

Manipular datos y hacer cálculos a veces es similar, pero normalmente es distinto a como lo haríamos en Pandas. De hecho, ya viste una diferencia: el uso del método `.show(n)`

Otras diferencias son:

In [4]:
# Conocer la dimension de mi DataFrame

print((df.count(), len(df.columns)))

(903871, 13)


In [14]:
# ¿Qué tipos de datos tengo?

df.printSchema()

root
 |-- ID_REGISTRO: string (nullable = true)
 |-- ENTIDAD_RES: string (nullable = true)
 |-- MUNICIPIO_RES: string (nullable = true)
 |-- FECHA_INGRESO: string (nullable = true)
 |-- FECHA_SINTOMAS: string (nullable = true)
 |-- covidt: string (nullable = true)
 |-- delta: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- alt: string (nullable = true)
 |-- qry: string (nullable = true)
 |-- dayofyear: string (nullable = true)
 |-- lengthofday: string (nullable = true)



### Tarea

Aplica los cambios siguientes:
1. Modifica tu DataFrame: agrega una columna nueva llamada "PAIS", en donde todas las hileras tendrán el valor "Mexico".

2. Elimina la columna *lengthofday* de tu DataFrame.

3. Crea un nuevo DataFrame, llamado `ubicacion`, que tendrá solamente los valores de *lat*, *lon*, *ENTIDAD_RES* y *PAIS* de `df`.

4. Filtra la columna *ENTIDAD_RES* dentro de `ubicacion` para quedarte con solo los valores = 9 (Ciudad de México). Guarda el resultado en un DataFrame llamado `cdmx`

5. Selecciona los primeros 5 valores de `cdmx` y guárdalos como un DataFrame de Pandas, con el nombre *df_pandas*.

####Tip
Métodos y Funciones que podrían servirte:

- .select()
- .drop()
- .withcolumn()
- lit()
- .filter()
- .limit()
- toPandas()



1. Modifica tu DataFrame: agrega una columna nueva llamada "PAIS", en donde todas las hileras tendrán el valor "Mexico".

Primero importamos la función lit junto con with Column para poder añadir la columna país y poner el en cada fila México

In [5]:
from pyspark.sql.functions import lit

In [6]:
df = df.withColumn("PAIS", lit ("Mexico"))
df.show(5)

+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+------+
|ID_REGISTRO|ENTIDAD_RES|MUNICIPIO_RES|FECHA_INGRESO|FECHA_SINTOMAS|covidt|delta|               lat|               long|               alt|                 qry|dayofyear|       lengthofday|  PAIS|
+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+------+
|     z526b3|          9|           12|   2020-12-21|    2020-12-18|     1|    3| 19.20155345588235| -99.20180252450989|3008.9460784313724|Cve_Ent==9 & Cve_...|      356|11.768371962176587|Mexico|
|     z3d1e2|          9|            5|   2020-04-22|    2020-04-20|     1|    2|         19.482945|         -99.113471|            2229.0|Cve_Ent==9 & Cve_...|      113|13.441805501598786|Mexico|
|     z21f6f|  

2. Elimina la columna lengthofday de tu DataFrame.

En esta parte se usará la función drop para poder eliminar dicha columna

In [7]:
df = df.drop('lengthofday')
df.show(5)

+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------+
|ID_REGISTRO|ENTIDAD_RES|MUNICIPIO_RES|FECHA_INGRESO|FECHA_SINTOMAS|covidt|delta|               lat|               long|               alt|                 qry|dayofyear|  PAIS|
+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------+
|     z526b3|          9|           12|   2020-12-21|    2020-12-18|     1|    3| 19.20155345588235| -99.20180252450989|3008.9460784313724|Cve_Ent==9 & Cve_...|      356|Mexico|
|     z3d1e2|          9|            5|   2020-04-22|    2020-04-20|     1|    2|         19.482945|         -99.113471|            2229.0|Cve_Ent==9 & Cve_...|      113|Mexico|
|     z21f6f|          7|            9|   2020-04-27|    2020-04-24|     1|    3| 16.21599257359306| -93.93758

3. Crea un nuevo DataFrame, llamado ubicacion, que tendrá solamente los valores de lat, lon, ENTIDAD_RES y PAIS de df.

En esta parte se utilizará la función select para crear el nuevo DataFrame

In [8]:
ubicacion = df.select("lat", "long", "ENTIDAD_RES", "PAIS")
ubicacion.show(5)

+------------------+-------------------+-----------+------+
|               lat|               long|ENTIDAD_RES|  PAIS|
+------------------+-------------------+-----------+------+
| 19.20155345588235| -99.20180252450989|          9|Mexico|
|         19.482945|         -99.113471|          9|Mexico|
| 16.21599257359306| -93.93758461688306|          7|Mexico|
| 26.06980561012655|-100.27776220506335|         19|Mexico|
|25.679631916666665|-100.16954308333334|         19|Mexico|
+------------------+-------------------+-----------+------+
only showing top 5 rows



4. Filtra la columna ENTIDAD_RES dentro de ubicacion para quedarte con solo los valores = 9 (Ciudad de México). Guarda el resultado en un DataFrame llamado cdmx

Ahora utilizando la función filter se busca solo quedarnos con los valores iguales a 9

In [9]:
cdmx = ubicacion.filter(ubicacion.ENTIDAD_RES == 9)
cdmx.show(5)

+-----------------+------------------+-----------+------+
|              lat|              long|ENTIDAD_RES|  PAIS|
+-----------------+------------------+-----------+------+
|19.20155345588235|-99.20180252450989|          9|Mexico|
|        19.482945|        -99.113471|          9|Mexico|
|        19.359004|        -99.092623|          9|Mexico|
|        19.395901|        -99.097613|          9|Mexico|
|19.20155345588235|-99.20180252450989|          9|Mexico|
+-----------------+------------------+-----------+------+
only showing top 5 rows



5. Selecciona los primeros 5 valores de cdmx y guárdalos como un DataFrame de Pandas, con el nombre df_pandas.

Por ultimo, utilizando limit vamor a seleccionar solo los primeros 5 valores de nuestro DataFrame cdmx y despues con toPandas lo convertiremos un DataFrame Pandas

In [10]:
cdmx_pandas= cdmx.limit(5).toPandas()
cdmx_pandas

,lat,long,ENTIDAD_RES,PAIS
0,19.20155345588235,-99.20180252450989,9,Mexico
1,19.482945,-99.113471,9,Mexico
2,19.359004,-99.092623,9,Mexico
3,19.395901,-99.097613,9,Mexico
4,19.20155345588235,-99.20180252450989,9,Mexico
